# Imports

In [ ]:
from lib.classical_classifiers import RBF_SVM, confusion_values, accuracy_from_confusion_values
import numpy as np
from lib.cloud_classification_data import get_scene, get_scene_mask, import_scene_names, unzeropad
from pathlib import Path
import pandas as pd
import glob

# Access training and test files

In [ ]:
base_path = Path('../38-Cloud_test')
scenes_path = base_path/'test_sceneids_38-Cloud.csv'
scene_names = import_scene_names(scenes_path)

# The training data files
# The training data is prepared with the superpixel segmentation technique.
# Each data point is obtained with superpixel segmentation from the full training data set.
# The features are: 
#    'R_med', 'G_med', 'B_med', 'Nir_med',
#    'R_avg', 'G_avg', 'B_avg', 'Nir_avg',
#    'R_iqr', 'G_iqr', 'B_iqr', 'Nir_iqr',
#    'R_max', 'G_max', 'B_max', 'Nir_max',
#    'R_min', 'G_min', 'B_min', 'Nir_min',
#    'R_std', 'G_std', 'B_std', 'Nir_std',
#    'gt_avg'

train_path = 'experiment_tr_data/*train*' 
train_files = sorted(glob.glob(train_path))
validate_files = [s.replace('train', 'validate') for s in train_files]

# Classification

### Preparation

In [ ]:
# Choose one file from training set
file_id = 0

# Choose hyperparameter range for the validation grid search
gamma_range = (0.1,150,3)# (min, max, step)
C_range = (0.1,150,3)# (min, max, step)

In [ ]:
# Get training data
print(file_id+1, ' of ', len(train_files))
train = pd.read_csv(train_files[file_id]).to_numpy()
x_train = train[:,:-1]
y_train = train[:,-1]

# Get validation data
validate = pd.read_csv(validate_files[file_id]).to_numpy()
x_validate = validate[:,:-1]
y_validate = validate[:,-1]

### Define classifier

In [ ]:
rbf = RBF_SVM(gamma = 1)

### Train classifier

In [ ]:
rbf.validate(x_train, y_train, x_validate, y_validate, C_range, gamma_range, info=True)

### Predict scene

Choose and import scene

In [ ]:
scene_id = 0

# Get scene data
scene_data, r_max, c_max, patches_ids_array, scene_names = get_scene(base_path, scene_id)

# Get scene ground-truth
gt_path = Path('../38-Cloud_test/Entire_scene_gts')
gt_scene = get_scene_mask(gt_path, scene_names, scene_id)

Perform prediction

In [ ]:
pred = rbf.predict_scene_sp(scene_data=scene_data, r_max=r_max, c_max=c_max, array_of_patches_indices=patches_ids_array, scene_name=scene_names[scene_id])

Compare predition with ground-truth

In [ ]:
# The prediction has to be trimmed to ground-truth size
trimmed_pr_gt = unzeropad(pred,gt_scene)

conf_val = confusion_values(trimmed_pr_gt, gt_scene)
rbf.export_scene_prediction_results(confusion_values=conf_val, dataset_name = 'SP')
print('Finished prediction! Scene: ', scene_names[scene_id])
print('Scene accurancy: ', accuracy_from_confusion_values(conf_val=conf_val))